In [58]:
import pandas as pd
import numpy as np

### 1. def PlayTimeGenre(genero: str): 

Debe devolver año con mas horas jugadas para dicho género

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [ ]:
#Leer el archivo con dataframe df_games desde el archivo tipo parquet
df_games_api1 = pd.read_parquet("./datasets/df_game_api.parquet")
print(df_games_api1.head(5))

In [ ]:
# Para convertir la columna year en variable de tipo entero
df_games_api1 = df_games_api1.dropna(subset=['year'])
df_games_api1['year'] = df_games_api1['year'].astype(int)
print(df_games_api1.info())

In [8]:
#Leer el archivo con dataframe df_items desde el archivo tipo parquet
df_items_api1 = pd.read_parquet("./datasets/df_items_api.parquet")
print(df_items_api1.head(1))

             user_id  item_id  playtime_forever
0  76561197970982479       10                 6


In [50]:
# Fusionar los 2 DataFrames 
df_api1_merged = pd.merge(df_games_api1, df_items_api1, on='item_id')
print(df_api1_merged.head(1))

                          genres                 title  item_id  year  \
0  ['action', 'indie', 'racing']  Carmageddon Max Pack   282010  1997   

    user_id  playtime_forever  
0  UTNerd24                 5  


In [ ]:
# Nos quedamos solo con las columnas requeridas para la API
df_api1_merged = df_api1_merged.drop(['title', 'item_id', 'user_id'], axis=1)
print(df_api1_merged.columns)

In [52]:
# Se genera el archivo que sera leido por la  API
df_api1_merged.to_parquet("./data/df_api1.parquet", index=False)

In [59]:
print(df_api1_merged.tail(100))

             genres  year  playtime_forever
2703555  ['action']  2004                 7
2703556  ['action']  2004                76
2703557  ['action']  2004                47
2703558  ['action']  2004                 6
2703559  ['action']  2004                 2
...             ...   ...               ...
2703650  ['action']  2004               203
2703651  ['action']  2004               244
2703652  ['action']  2004                 7
2703653  ['action']  2004                 9
2703654  ['action']  2004                99

[100 rows x 3 columns]


In [60]:
conteo_valores = df_api1_merged['genres'].value_counts()
print(conteo_valores)

genres
['action']                                                                  487157
['action', 'adventure']                                                     196173
['action', 'indie']                                                         119943
['strategy']                                                                117890
['action', 'rpg']                                                           108480
                                                                             ...  
['indie', 'massively multiplayer', 'strategy', 'early access']                   1
['action', 'free to play', 'indie', 'massively multiplayer', 'strategy']         1
['adventure', 'early access']                                                    1
['education']                                                                    1
['casual', 'indie', 'racing', 'sports']                                          1
Name: count, Length: 565, dtype: int64


In [62]:
# Funcion para resolver problema API NO. 1: PlayTimeGenre
def playTimeGenre(genero:str):

    # Se cambia a letra minuscula el valor de genero ingresado
    df_games['genres'] = df_games['genres'].astype(str).str.lower()
    genero = genero.lower()

    # Filtramos el dataframe con solo las filas que contengan el genero buscado (que est dentro de una lista en la columna genres)
    def buscar_genero(genres_lista, genero_buscar):
        return genero_buscar in genres_lista

    df_api1_genres = df_api1_merged[df_api1_merged['genres'].apply(buscar_genero, genero_buscar=genero)]
    #print(df_api1_genres.head())

    # Agrupar por año y sumar las horas jugadas
    df_api1_grouped = df_api1_genres.groupby('year')['playtime_forever'].sum()
    #print(df_api1_grouped)

    # Encontrar el año con la suma máxima de horas jugadas
    anio_max_horas_jugadas = df_api1_grouped.idxmax()

    resultado=f"Año de lanzamiento con más horas jugadas para Género {genero} es: {anio_max_horas_jugadas}"
        
    return resultado

playTimeGenre('Strategy')


ValueError: attempt to get argmax of an empty sequence